In [1]:
import os
from datetime import UTC, datetime

from rich import print as rprint

from entitysdk import Client, ProjectContext, models

entitycore_api_url = "http://127.0.0.1:8000"
project_context = ProjectContext(
    virtual_lab_id="a98b7abc-fc46-4700-9e3d-37137812c730",
    project_id="0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6",
)
token = os.getenv("ACCESS_TOKEN", "XXX")
client = Client(api_url=entitycore_api_url, project_context=project_context, token_manager=token)

## Register a Circuit

In [2]:
species = client.search_entity(
    entity_type=models.Species, query={"name__ilike": "Homo sapiens"}
).one()

subject = models.Subject(
    name="my-subject", description="my-subject-description", sex="male", species=species
)
subject = client.register_entity(subject)

brain_region = client.search_entity(entity_type=models.BrainRegion, query={"acronym": "CB"}).first()


circuit = models.Circuit(
    name="my-circuit",
    description="my-circuit",
    subject=subject,
    brain_region=brain_region,
    number_synapses=2,
    number_neurons=5,
    number_connections=10,
    scale="microcircuit",
    build_category="em_reconstruction",
)

circuit = client.register_entity(circuit)

# 1. Register the entire provenance at once
When all entities are available up front the entire provenance can be generated using the creation endpoints without any updating taking place.

## Register all entities

In [4]:
campaign = client.register_entity(
    models.SimulationCampaign(
        name="my-campaign",
        description="my-campaign-description",
        entity_id=circuit.id,
        scan_parameters={"foo": "bar"},
    )
)

simulations = [
    client.register_entity(
        models.Simulation(
            name=f"sim-{i}",
            description=f"sim-{i}",
            scan_parameters={"foo": "bar"},
            entity_id=circuit.id,
            simulation_campaign_id=campaign.id,
        )
    )
    for i in range(5)
]
simulation_results = [
    client.register_entity(
        models.SimulationResult(
            name=f"result-{i}",
            description=f"result-{i}",
            simulation_id=simulations[i].id,
        )
    )
    for i in range(5)
]

## Register activities

In [5]:
simulation_generation = client.register_entity(
    models.SimulationGeneration(
        start_time=datetime.now(UTC),
        used=[campaign],
        generated=simulations,
    )
)

simulation_executions = [
    client.register_entity(
        models.SimulationExecution(
            used=[simulations[i]],
            generated=[simulation_results[i]],
            start_time=datetime.now(UTC),
            status="done",
        )
    )
    for i in range(5)
]

In [6]:
rprint(simulation_generation)
rprint(simulation_executions)

SimulationGeneration(
    id=UUID('03b5d1ee-0441-416b-8cd5-a442d787b3a0'),
    update_date=datetime.datetime(2025, 9, 19, 14, 8, 6, 206765, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 9, 19, 14, 8, 6, 206765, tzinfo=TzInfo(UTC)),
    type=<ActivityType.simulation_generation: 'simulation_generation'>,
    start_time=datetime.datetime(2025, 9, 19, 14, 8, 6, 169161, tzinfo=TzInfo(UTC)),
    end_time=None,
    used=[
        Entity(
            assets=[],
            id=UUID('78400cb6-9cb2-42e4-b8da-6b5ee017ee73'),
            update_date=None,
            creation_date=None,
            name=None,
            description=None,
            type=<EntityType.simulation_campaign: 'simulation_campaign'>,
            created_by=None,
            updated_by=None,
            authorized_public=False,
            authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
            contributions=None,
            legacy_id=None
        )
    ],
    generated=[
        Entity(
            assets=[],
            id=UUID('920c9740-d3bf-4ee2-98de-9442b31443f5'),
            update_date=None,
            creation_date=None,
            name=None,
            description=None,
            type=<EntityType.simulation: 'simulation'>,
            created_by=None,
            updated_by=None,
            authorized_public=False,
            authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
            contributions=None,
            legacy_id=None
        ),
        Entity(
            assets=[],
            id=UUID('a84a474d-f2b6-435f-aa5a-ff1ecb2430f7'),
            update_date=None,
            creation_date=None,
            name=None,
            description=None,
            type=<EntityType.simulation: 'simulation'>,
            created_by=None,
            updated_by=None,
            authorized_public=False,
            authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
            contributions=None,
            legacy_id=None
        ),
        Entity(
            assets=[],
            id=UUID('e645674a-088e-4cf4-91ad-bfce45c087d1'),
            update_date=None,
            creation_date=None,
            name=None,
            description=None,
            type=<EntityType.simulation: 'simulation'>,
            created_by=None,
            updated_by=None,
            authorized_public=False,
            authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
            contributions=None,
            legacy_id=None
        ),
        Entity(
            assets=[],
            id=UUID('0a00b021-717e-4b9a-a20f-6942cf2fbcaf'),
            update_date=None,
            creation_date=None,
            name=None,
            description=None,
            type=<EntityType.simulation: 'simulation'>,
            created_by=None,
            updated_by=None,
            authorized_public=False,
            authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
            contributions=None,
            legacy_id=None
        ),
        Entity(
            assets=[],
            id=UUID('6764b5ab-bb4c-404c-ac20-201f8c122adb'),
            update_date=None,
            creation_date=None,
            name=None,
            description=None,
            type=<EntityType.simulation: 'simulation'>,
            created_by=None,
            updated_by=None,
            authorized_public=False,
            authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
            contributions=None,
            legacy_id=None
        )
    ]
)

[
    SimulationExecution(
        id=UUID('0c0041fd-2002-4324-a928-9ceef5e3a8b1'),
        update_date=datetime.datetime(2025, 9, 19, 14, 8, 6, 400135, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 9, 19, 14, 8, 6, 400135, tzinfo=TzInfo(UTC)),
        type=<ActivityType.simulation_execution: 'simulation_execution'>,
        start_time=datetime.datetime(2025, 9, 19, 14, 8, 6, 365189, tzinfo=TzInfo(UTC)),
        end_time=None,
        used=[
            Entity(
                assets=[],
                id=UUID('920c9740-d3bf-4ee2-98de-9442b31443f5'),
                update_date=None,
                creation_date=None,
                name=None,
                description=None,
                type=<EntityType.simulation: 'simulation'>,
                created_by=None,
                updated_by=None,
                authorized_public=False,
                authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
                contributions=None,
                legacy_id=None
            )
        ],
        generated=[
            Entity(
                assets=[],
                id=UUID('b1069b5a-4e0b-4824-88d1-c0e9eced6916'),
                update_date=None,
                creation_date=None,
                name=None,
                description=None,
                type=<EntityType.simulation_result: 'simulation_result'>,
                created_by=None,
                updated_by=None,
                authorized_public=False,
                authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
                contributions=None,
                legacy_id=None
            )
        ],
        status=<SimulationExecutionStatus.done: 'done'>
    ),
    SimulationExecution(
        id=UUID('c2293f2a-ec2f-49f6-a62f-dee49def5204'),
        update_date=datetime.datetime(2025, 9, 19, 14, 8, 6, 413778, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 9, 19, 14, 8, 6, 413778, tzinfo=TzInfo(UTC)),
        type=<ActivityType.simulation_execution: 'simulation_execution'>,
        start_time=datetime.datetime(2025, 9, 19, 14, 8, 6, 378864, tzinfo=TzInfo(UTC)),
        end_time=None,
        used=[
            Entity(
                assets=[],
                id=UUID('a84a474d-f2b6-435f-aa5a-ff1ecb2430f7'),
                update_date=None,
                creation_date=None,
                name=None,
                description=None,
                type=<EntityType.simulation: 'simulation'>,
                created_by=None,
                updated_by=None,
                authorized_public=False,
                authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
                contributions=None,
                legacy_id=None
            )
        ],
        generated=[
            Entity(
                assets=[],
                id=UUID('6d9bb151-077f-4413-92f5-d385523a97de'),
                update_date=None,
                creation_date=None,
                name=None,
                description=None,
                type=<EntityType.simulation_result: 'simulation_result'>,
                created_by=None,
                updated_by=None,
                authorized_public=False,
                authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
                contributions=None,
                legacy_id=None
            )
        ],
        status=<SimulationExecutionStatus.done: 'done'>
    ),
    SimulationExecution(
        id=UUID('66c7b860-5fd3-4827-955e-ddd6a011414b'),
        update_date=datetime.datetime(2025, 9, 19, 14, 8, 6, 422209, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 9, 19, 14, 8, 6, 422209, tzinfo=TzInfo(UTC)),
        type=<ActivityType.simulation_execution: 'simulation_execution'>,
        start_time=datetime.datetime(2025, 9, 19, 14, 8, 6, 388066, tzinfo=TzInfo(UTC)),
        end_time=None,
        used=[
            Entity(
                assets=[],
                id

# 2. Register provenance gradually

## Register a SimulationCampaign

In [7]:
campaign = client.register_entity(
    models.SimulationCampaign(
        name="my-campaign",
        description="my-campaign-description",
        entity_id=circuit.id,
        scan_parameters={"foo": "bar"},
    )
)

## Register SimulationGeneration

In [8]:
sim_generation = client.register_entity(
    models.SimulationGeneration(
        start_time=datetime.now(UTC),
        used=[campaign],
    )
)

## Register Simulations

In [9]:
simulations = [
    client.register_entity(
        models.Simulation(
            name=f"sim-{i}",
            description=f"sim-{i}",
            scan_parameters={"foo": "bar"},
            entity_id=circuit.id,
            simulation_campaign_id=campaign.id,
        )
    )
    for i in range(5)
]

## Update SimulationGeneration with simulations

In [10]:
data_to_update = {
    "generated_ids": [s.id for s in simulations],
    "end_time": datetime.now(UTC),
}

updated_sim_generation = client.update_entity(
    entity_id=sim_generation.id, entity_type=type(sim_generation), attrs_or_entity=data_to_update
)

In [11]:
rprint(updated_sim_generation)

SimulationGeneration(
    id=UUID('7518d19f-1f73-4605-83b6-bc8dfd76a3f0'),
    update_date=datetime.datetime(2025, 9, 19, 14, 8, 15, 889862, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 9, 19, 14, 8, 12, 375633, tzinfo=TzInfo(UTC)),
    type=<ActivityType.simulation_generation: 'simulation_generation'>,
    start_time=datetime.datetime(2025, 9, 19, 14, 8, 12, 338255, tzinfo=TzInfo(UTC)),
    end_time=datetime.datetime(2025, 9, 19, 14, 8, 15, 840151, tzinfo=TzInfo(UTC)),
    used=[
        Entity(
            assets=[],
            id=UUID('adf693b2-0e10-4a9c-8b99-3b84ee5df83c'),
            update_date=None,
            creation_date=None,
            name=None,
            description=None,
            type=<EntityType.simulation_campaign: 'simulation_campaign'>,
            created_by=None,
            updated_by=None,
            authorized_public=False,
            authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
            contributions=None,
            legacy_id=None
        )
    ],
    generated=[
        Entity(
            assets=[],
            id=UUID('07198aa8-5019-46d5-845a-4af1d3f1b6e1'),
            update_date=None,
            creation_date=None,
            name=None,
            description=None,
            type=<EntityType.simulation: 'simulation'>,
            created_by=None,
            updated_by=None,
            authorized_public=False,
            authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
            contributions=None,
            legacy_id=None
        ),
        Entity(
            assets=[],
            id=UUID('7152811c-c099-4562-b709-4b4fb82f6034'),
            update_date=None,
            creation_date=None,
            name=None,
            description=None,
            type=<EntityType.simulation: 'simulation'>,
            created_by=None,
            updated_by=None,
            authorized_public=False,
            authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
            contributions=None,
            legacy_id=None
        ),
        Entity(
            assets=[],
            id=UUID('0bf4899e-de41-4f8e-923d-a3c8764f92a8'),
            update_date=None,
            creation_date=None,
            name=None,
            description=None,
            type=<EntityType.simulation: 'simulation'>,
            created_by=None,
            updated_by=None,
            authorized_public=False,
            authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
            contributions=None,
            legacy_id=None
        ),
        Entity(
            assets=[],
            id=UUID('627961e5-78fb-48fc-aeac-2f6f829126ea'),
            update_date=None,
            creation_date=None,
            name=None,
            description=None,
            type=<EntityType.simulation: 'simulation'>,
            created_by=None,
            updated_by=None,
            authorized_public=False,
            authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
            contributions=None,
            legacy_id=None
        ),
        Entity(
            assets=[],
            id=UUID('6feba4b4-ba4f-4673-b02b-b37185a30cc9'),
            update_date=None,
            creation_date=None,
            name=None,
            description=None,
            type=<EntityType.simulation: 'simulation'>,
            created_by=None,
            updated_by=None,
            authorized_public=False,
            authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
            contributions=None,
            legacy_id=None
        )
    ]
)

## Create simulation executions

In [12]:
executions = [
    client.register_entity(
        models.SimulationExecution(
            used=[simulations[i]],
            start_time=datetime.now(UTC),
            status="created",
        )
    )
    for i in range(5)
]

## Create simulation results

In [14]:
results = [
    client.register_entity(
        models.SimulationResult(
            name=f"result-{i}",
            description=f"result-{i}",
            simulation_id=simulations[i].id,
        )
    )
    for i in range(5)
]

## Update simulation executions with results

In [15]:
updated_executions = [
    client.update_entity(
        entity_id=executions[i].id,
        entity_type=models.SimulationExecution,
        attrs_or_entity={
            "generated_ids": [results[i].id],
            "end_time": datetime.now(UTC),
            "status": "done",
        },
    )
    for i in range(5)
]

In [16]:
rprint(updated_executions)

[
    SimulationExecution(
        id=UUID('f347223d-53a1-4c5f-8296-f318ca1b3cab'),
        update_date=datetime.datetime(2025, 9, 19, 14, 9, 8, 621517, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 9, 19, 14, 8, 18, 809323, tzinfo=TzInfo(UTC)),
        type=<ActivityType.simulation_execution: 'simulation_execution'>,
        start_time=datetime.datetime(2025, 9, 19, 14, 8, 18, 767168, tzinfo=TzInfo(UTC)),
        end_time=datetime.datetime(2025, 9, 19, 14, 9, 8, 569547, tzinfo=TzInfo(UTC)),
        used=[
            Entity(
                assets=[],
                id=UUID('07198aa8-5019-46d5-845a-4af1d3f1b6e1'),
                update_date=None,
                creation_date=None,
                name=None,
                description=None,
                type=<EntityType.simulation: 'simulation'>,
                created_by=None,
                updated_by=None,
                authorized_public=False,
                authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
                contributions=None,
                legacy_id=None
            )
        ],
        generated=[
            Entity(
                assets=[],
                id=UUID('3e8af732-3263-4b38-9e90-df957c123694'),
                update_date=None,
                creation_date=None,
                name=None,
                description=None,
                type=<EntityType.simulation_result: 'simulation_result'>,
                created_by=None,
                updated_by=None,
                authorized_public=False,
                authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
                contributions=None,
                legacy_id=None
            )
        ],
        status=<SimulationExecutionStatus.done: 'done'>
    ),
    SimulationExecution(
        id=UUID('bca969fc-06fa-41cf-b8fc-bedc45c7d2f5'),
        update_date=datetime.datetime(2025, 9, 19, 14, 9, 8, 657809, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 9, 19, 14, 8, 18, 836499, tzinfo=TzInfo(UTC)),
        type=<ActivityType.simulation_execution: 'simulation_execution'>,
        start_time=datetime.datetime(2025, 9, 19, 14, 8, 18, 801214, tzinfo=TzInfo(UTC)),
        end_time=datetime.datetime(2025, 9, 19, 14, 9, 8, 622199, tzinfo=TzInfo(UTC)),
        used=[
            Entity(
                assets=[],
                id=UUID('7152811c-c099-4562-b709-4b4fb82f6034'),
                update_date=None,
                creation_date=None,
                name=None,
                description=None,
                type=<EntityType.simulation: 'simulation'>,
                created_by=None,
                updated_by=None,
                authorized_public=False,
                authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
                contributions=None,
                legacy_id=None
            )
        ],
        generated=[
            Entity(
                assets=[],
                id=UUID('8002aea1-c48b-44d1-bbf7-81d401d061cc'),
                update_date=None,
                creation_date=None,
                name=None,
                description=None,
                type=<EntityType.simulation_result: 'simulation_result'>,
                created_by=None,
                updated_by=None,
                authorized_public=False,
                authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
                contributions=None,
                legacy_id=None
            )
        ],
        status=<SimulationExecutionStatus.done: 'done'>
    ),
    SimulationExecution(
        id=UUID('2b860361-2509-4a63-82f7-46f2c7038d6e'),
        update_date=datetime.datetime(2025, 9, 19, 14, 9, 8, 673486, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 9, 19, 14, 8, 18, 851946, tzinfo=TzInfo(UTC)),
        type=<ActivityType.simulation_execution: 'simulation_execution'>,
        start_time=datetime.datetime(2025, 9, 19, 14, 8, 18